In [144]:
import pandas as pd
import numpy as np
import os
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [357]:
# !pip install python-dotenv

In [3]:
!pwd

/home/lucky/Documents/projects/Zywa_Backend/experiments


In [5]:
os.chdir("../")

In [10]:
!pwd

/home/lucky/Documents/projects/Zywa_Backend


In [362]:
from dotenv import load_dotenv
load_dotenv()  # Loads variables from the .env file
# os.getenv('DATABASE_URL')

True

In [363]:
uri = os.getenv('DATABASE_URL')

In [150]:
os.chdir("data/")

FileNotFoundError: [Errno 2] No such file or directory: 'data/'

In [16]:
files = os.listdir()
files

['Sample Card Status Info - Delivered.csv',
 'Sample Card Status Info - Delivery exceptions.csv',
 'Sample Card Status Info - Returned.csv',
 'Sample Card Status Info - Pickup.csv']

In [20]:
"DELIVERED" in files[0].upper()

True

In [27]:
"EXCEPTIONS" in files[1].upper()

True

In [24]:
"RETURNED" in files[2].upper()

True

In [26]:
"PICKUP" in files[3].upper()

True

In [28]:
labeled_files = {"DELIVERED" : [],
                 "EXCEPTIONS" : [],
                 "RETURNED" : [],
                 "PICKUP" : []}

for file in files:
    if("DELIVERED" in file.upper()):
        labeled_files['DELIVERED'].append(file)
    elif("EXCEPTIONS" in file.upper()):
        labeled_files['EXCEPTIONS'].append(file)
    elif("RETURNED" in file.upper()):
        labeled_files['RETURNED'].append(file)
    elif("PICKUP" in file.upper()):
        labeled_files['PICKUP'].append(file)

In [29]:
labeled_files

{'DELIVERED': ['Sample Card Status Info - Delivered.csv'],
 'EXCEPTIONS': ['Sample Card Status Info - Delivery exceptions.csv'],
 'RETURNED': ['Sample Card Status Info - Returned.csv'],
 'PICKUP': ['Sample Card Status Info - Pickup.csv']}

In [32]:
pd.read_csv(labeled_files['DELIVERED'][0])

,ID,Card ID,User contact,Timestamp,Comment
0,A883,ZYW8827,"""0585949014""",2023-11-13T09:34:56Z,DELIVERED
1,A073,ZYW7631,"""0545576586""",2023-11-14T12:34:56Z,DELIVERED


In [33]:
pd.read_csv(labeled_files['EXCEPTIONS'][0])

,ID,Card ID,User contact,Timestamp,Comment
0,A982,ZYW8890,"""0534534534""",13-11-2023 14:00,User not available
1,A073,ZYW7631,"""0545576586""",13-11-2023 16:00,User declined to accept package


In [34]:
pd.read_csv(labeled_files['RETURNED'][0])

,ID,Card ID,User contact,Timestamp
0,A982,ZYW8890,971534534534,14-11-2023 12:00PM


In [35]:
pd.read_csv(labeled_files['PICKUP'][0])

,ID,Card ID,User Mobile,Timestamp
0,332,ZYW8827,585949014,12-11-2023 11:59 PM
1,465,ZYW8890,534534534,12-11-2023 3:45 AM
2,878,ZYW7631,545576586,13-11-2023 11:02 AM


### creating card df

In [39]:
column_names = ['CARD_ID', 'USER_CONTACT', 'STATUS', 'TIMESTAMP', 'COMMENT']
card_df = pd.DataFrame(columns=column_names)
card_df

,CARD_ID,USER_CONTACT,STATUS,TIMESTAMP,COMMENT


### RegEx code for removing outer inverted commas

In [76]:
import re

def remove_outer_inverted_commas(text):
  """
  Removes inverted commas (") from the beginning and end of a string, if present.

  Args:
    text: The string to process.

  Returns:
    The string with the outer inverted commas removed, if present. Otherwise, the original string.
  """

  # Use regular expressions to match and remove outer inverted commas
  pattern = r'^"(.*?)"$'
  match = re.match(pattern, text)

  if match:
    return match.group(1)  # Return the inner content
  else:
    return text  # No change if no outer inverted commas

# Test cases
strings = [
  '"This is a string with outer inverted commas."',
  'This is a string without outer inverted commas.',
  '"Only the first one should be removed."',
  'Both "outer" and "inner" inverted commas should be removed.',
  '"',
  ''
]

for string in strings:
  print(f"Original: {string}")
  result = remove_outer_inverted_commas(string)
  print(f"Result: {result}")


Original: "This is a string with outer inverted commas."
Result: This is a string with outer inverted commas.
Original: This is a string without outer inverted commas.
Result: This is a string without outer inverted commas.
Original: "Only the first one should be removed."
Result: Only the first one should be removed.
Original: Both "outer" and "inner" inverted commas should be removed.
Result: Both "outer" and "inner" inverted commas should be removed.
Original: "
Result: "
Original: 
Result: 


### rename columns name

In [335]:
def process_dataframe(df, status):
    df = df.rename(columns={'Card ID': 'CARD_ID', 'User contact' : "USER_CONTACT", "User Mobile" : "USER_CONTACT","Timestamp":"TIMESTAMP", "Comment" : "COMMENT"})
    # appling regex only to object types columns
    df[df.select_dtypes(include=['object']).columns.values] = df.select_dtypes(include=['object']).map(remove_outer_inverted_commas)
    df["USER_CONTACT"] = df["USER_CONTACT"].astype('int64')

    # pick up last 9 digits of contant number
    df["USER_CONTACT"] = df["USER_CONTACT"].map(lambda x : str(x)[-9:])
    df["STATUS"] = status

    def parse_flexible_format(timestamp):
        for fmt in ('%d-%m-%Y %I:%M %p', '%d-%m-%Y %I:%M%p', '%d-%m-%Y %H:%M','%Y-%m-%dT%H:%M:%SZ'):  # Add more formats if needed
            try:
                return datetime.strptime(timestamp, fmt)
            except ValueError:
                pass  # Try the next format
        raise ValueError(f"Unrecognized timestamp format: {timestamp}") 
    
    # Apply the appropriate parsing function based on the format
    df['TIMESTAMP'] = df['TIMESTAMP'].apply(parse_flexible_format)
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    return df

In [336]:
df = pd.read_csv(labeled_files['EXCEPTIONS'][0])
df

,ID,Card ID,User contact,Timestamp,Comment
0,A982,ZYW8890,"""0534534534""",13-11-2023 14:00,User not available
1,A073,ZYW7631,"""0545576586""",13-11-2023 16:00,User declined to accept package


In [337]:
df = process_dataframe(df, "EXCEPTIONS")
df

,ID,CARD_ID,USER_CONTACT,TIMESTAMP,COMMENT,STATUS
0,A982,ZYW8890,534534534,2023-11-13 14:00:00,User not available,EXCEPTIONS
1,A073,ZYW7631,545576586,2023-11-13 16:00:00,User declined to accept package,EXCEPTIONS


## Database connected

In [156]:
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [154]:
# Close the connection
client.close()

In [240]:
db = client["mydb"]
collection = db["zywa"]


In [163]:
document = {
    "CARD_ID": "ABC123",
    "USER_CONTACT": 1234567890,
    "STATUS_HISTORY": [
        {
            "STATUS": "pending",
            "COMMENT": "Initial status",
            "TIMESTAMP": "11-04-2014"
        }
    ]
}

collection.insert_one(document) 

InsertOneResult(ObjectId('65da37a1e1943bc1e996f706'), acknowledged=True)

### Update the status

In [164]:
card_id = "ABC123"
new_status_entry = {
            "STATUS": "delivered",
            "COMMENT": "updated",
            "TIMESTAMP": "11-04-2017"
        }

collection.update_one(
    {"CARD_ID": card_id},
    {"$push": {"STATUS_HISTORY": new_status_entry}}
)

UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff000000000000019b'), 'opTime': {'ts': Timestamp(1708799929, 10), 't': 411}, 'nModified': 1, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708799929, 10), 'signature': {'hash': b'\x87\x08\x9aP)\x1a}4\x9a\xb09\x01Gy\x1c\xcc0\xc3\x0ey', 'keyId': 7306560211760709648}}, 'operationTime': Timestamp(1708799929, 10), 'updatedExisting': True}, acknowledged=True)

### Get data

In [168]:
card_id = "ABC123"  # The CARD_ID you want to search for
user_contact = 1234567890
result = collection.find_one({"USER_CONTACT": user_contact})

if result:
    print(result) 
else:
    print("Document with CARD_ID:", card_id, "not found")

{'_id': ObjectId('65da37a1e1943bc1e996f706'), 'CARD_ID': 'ABC123', 'USER_CONTACT': 1234567890, 'STATUS_HISTORY': [{'STATUS': 'pending', 'COMMENT': 'Initial status', 'TIMESTAMP': '11-04-2014'}, {'STATUS': 'delivered', 'COMMENT': 'updated', 'TIMESTAMP': '11-04-2017'}]}


## converting to class

In [245]:
import pymongo
from datetime import datetime

# --- Configuration (replace with your connection details) ---
# client = pymongo.MongoClient("your_mongodb_connection_string")
# db = client["your_database_name"]
# collection = db["your_collection_name"]


# --- Card Model ---
class Card:
    def __init__(self, CARD_ID, USER_CONTACT, STATUS_HISTORY, _id=None):
        self.CARD_ID = CARD_ID
        self.USER_CONTACT = USER_CONTACT
        self.STATUS_HISTORY = STATUS_HISTORY

    def save(self):
        """Inserts a new Card document into the MongoDB collection."""
        document = {
            "CARD_ID": self.CARD_ID,
            "USER_CONTACT": self.USER_CONTACT,
            "STATUS_HISTORY": self.STATUS_HISTORY
        }
        result = collection.insert_one(document)
        return result.inserted_id  # Optionally return the inserted ID

    @classmethod
    def find_by_card_id(cls, CARD_ID):
        """Finds a Card document by its CARD_ID."""
        document = collection.find_one({"CARD_ID": CARD_ID})
        if document:
            return cls(**document)  # Create a Card object from the document
        else:
            return None

    @classmethod
    def find_by_contant(cls, contact):
        """Finds a Card document by its CARD_ID."""
        document = collection.find_one({"USER_CONTACT": contact})
        if document:
            return cls(**document)  # Create a Card object from the document
        else:
            return None

    def update_status(self, STATUS, COMMENT, TIMESTAMP):
        """Appends a new status entry to the STATUS_HISTORY."""
        new_entry = {
            "STATUS": STATUS,
            "COMMENT": COMMENT,
            "TIMESTAMP": TIMESTAMP
        }
        result = collection.update_one(
            {"CARD_ID": self.CARD_ID},
            {"$push": {"STATUS_HISTORY": new_entry}}
        )
        return result.modified_count  # Returns the number of modified documents


In [185]:
# --- Example Usage ---
# Create a new card
card = Card("XYZ2278", 987432, [
    {"STATUS": "pending", "COMMENT": "Initial status", "TIMESTAMP": "2023-12-18T15:30:00Z"}
])
card.save()

ObjectId('65da3b9fe1943bc1e996f70c')

In [188]:
# Find a card by CARD_ID
found_card = Card.find_by_card_id("XYZ2278")
found_card
if found_card:
    found_card.update_status("updaasdted", "Status d for a reason", "2023-12-18T15:30:00Z")


In [189]:
found_card.STATUS_HISTORY

[{'STATUS': 'pending',
  'COMMENT': 'Initial status',
  'TIMESTAMP': '2023-12-18T15:30:00Z'},
 {'STATUS': 'updated',
  'COMMENT': 'Status changed for a reason',
  'TIMESTAMP': '2023-12-18T15:30:00Z'}]

### Create real data to database

In [339]:
df = pd.read_csv(labeled_files['PICKUP'][0])
df = process_dataframe(df, "PICKUP")
df

,ID,CARD_ID,USER_CONTACT,TIMESTAMP,STATUS
0,332,ZYW8827,585949014,2023-11-12 23:59:00,PICKUP
1,465,ZYW8890,534534534,2023-11-12 03:45:00,PICKUP
2,878,ZYW7631,545576586,2023-11-13 11:02:00,PICKUP


In [340]:
df.iloc[0].to_dict()

{'ID': 332,
 'CARD_ID': 'ZYW8827',
 'USER_CONTACT': '585949014',
 'TIMESTAMP': Timestamp('2023-11-12 23:59:00'),
 'STATUS': 'PICKUP'}

In [341]:
"ID" in df.iloc[0].to_dict().keys()

True

## ADD DATA FOR PICKUP ONLY

In [342]:
for i in range(len(df)):
    CARD_ID = df.iloc[i]["CARD_ID"]
    USER_CONTACT = df.iloc[i]["USER_CONTACT"]
    comment =  df.iloc[i]["COMMENT"] if "COMMENT" in df.iloc[i].to_dict().keys() else ""
    STATUS_HISTORY = [{"STATUS":  df.iloc[i]["STATUS"], "COMMENT": comment, "TIMESTAMP": df.iloc[i]["TIMESTAMP"]}]
    
    
    card = Card(CARD_ID, USER_CONTACT, STATUS_HISTORY)
    card.save()

## UPDATE database for restof the status

In [347]:
df = pd.read_csv(labeled_files['RETURNED'][0])
df = process_dataframe(df, "RETURNED")
df

,ID,CARD_ID,USER_CONTACT,TIMESTAMP,STATUS
0,A982,ZYW8890,534534534,2023-11-14 12:00:00,RETURNED


In [348]:
for i in range(len(df)):
    CARD_ID = df.iloc[i]["CARD_ID"]
    USER_CONTACT = df.iloc[i]["USER_CONTACT"]
    comment =  df.iloc[i]["COMMENT"] if "COMMENT" in df.iloc[i].to_dict().keys() else ""
    STATUS_HISTORY = {"STATUS":  df.iloc[i]["STATUS"], "COMMENT": comment, "TIMESTAMP": df.iloc[i]["TIMESTAMP"]}
    
    
    # Find a card by CARD_ID
    found_card = Card.find_by_card_id(CARD_ID)
    found_card
    if found_card:
        found_card.update_status(**STATUS_HISTORY)

## get the card info

In [352]:
card = Card.find_by_card_id("ZYW8890")
card.STATUS_HISTORY

[{'STATUS': 'PICKUP',
  'COMMENT': '',
  'TIMESTAMP': datetime.datetime(2023, 11, 12, 3, 45)},
 {'STATUS': 'EXCEPTIONS',
  'COMMENT': 'User not available',
  'TIMESTAMP': datetime.datetime(2023, 11, 13, 14, 0)},
 {'STATUS': 'RETURNED',
  'COMMENT': '',
  'TIMESTAMP': datetime.datetime(2023, 11, 14, 12, 0)}]

In [365]:
card.CARD_ID, card.USER_CONTACT

('ZYW8890', '534534534')

In [353]:
latest_dict = max(card.STATUS_HISTORY, key=lambda x: x['TIMESTAMP'])

print(latest_dict)

{'STATUS': 'RETURNED', 'COMMENT': '', 'TIMESTAMP': datetime.datetime(2023, 11, 14, 12, 0)}


In [354]:
# your_api_project/
# ├── app.py                     # Main application file
# ├── config.py                  # Configuration settings
# ├── models/                    # Data models and MongoDB interactions
# │   ├── __init__.py           
# │   └── card.py                # Example model
# ├── routes/                    # API endpoints (routes)
# │   ├── __init__.py           
# │   └── cards.py               # Example routes
# ├── tests/                     # Unit and integration tests
# │   ├── __init__.py           
# │   └── test_cards.py          # Example test file
# ├── requirements.txt           # Project dependencies
# └── README.md                  # Project description and instructions

In [367]:
db['zywa'].drop()